In [1]:
# https://search.mediacloud.org/
# https://pypi.org/project/mediacloud-metadata/
# https://github.com/mediacloud/mc-providers/blob/main/mc_providers/test/test_onlinenews.py#L272

import datetime
import json
import pandas as pd
import requests
from mcmetadata import extract

import mediacloud.api

# Init the connection
mc = mediacloud.api.DirectoryApi('56196a395ee77c33a296073fa08e72f541362a10')

# Read the query.csv file to obtain information of each item to dump
df_query = pd.read_csv('query_v3.csv')


In [2]:
size_query = df_query.shape[0]
print(size_query)

4


In [3]:
# Create the result dataframe
df_result = pd.DataFrame(columns=['year', 'title', 'url', 'text_content'])

for counter in range(0,size_query) :
    # Query the MediaCloud API with each item
    url_data = df_query.iloc[counter].url
    try: 
        metadata = extract(url=url_data)
        metadata['publication_date'] = metadata['publication_date'].year
    
        # Append data to the result dataframe
        list_metadata = []
        for field in ['publication_date', 'article_title', 'url', 'text_content'] :
            list_metadata.append(metadata[field])
        # Append data to the result dataframe
        df_result.loc[counter] = list_metadata
        # And wirte date in a JSON File
        with open('data.json', 'a') as f:
            json.dump(metadata, f)
            f.write('\n')
        
    except Exception as e:
        print(f'Erreur -> "{e}", pour l\'url : {url_data}' )
        log_message = f"{datetime.datetime.now()} - {e} ; {url_data}\n"
        with open('log.txt', 'a') as log_file:
            log_file.write(log_message)

print(df_result.describe)

Erreur -> "Webpage didn't return content (404) from https://www.airforcetimes.com/intel-geoint/isr/2022/06/03/unlocking-airborne-isr-can-help-achieve-regional-security-in-indo-pacific/", pour l'url : https://www.airforcetimes.com/intel-geoint/isr/2022/06/03/unlocking-airborne-isr-can-help-achieve-regional-security-in-indo-pacific/
<bound method NDFrame.describe of    year                                              title  \
0  2022  US military creates space unit in South Korea ...   
1  2022  China using civilian ships to enhance navy cap...   
3  2022  Burn pit registry is failing to help track vet...   

                                                 url  \
0  https://www.airforcetimes.com/battlefield-tech...   
1  https://www.airforcetimes.com/flashpoints/chin...   
3  https://www.airforcetimes.com/news/burn-pits/2...   

                                        text_content  
0  SEOUL, South Korea — The U.S. military formall...  
1  BANGKOK — A Chinese scientific ship bristling 

In [4]:
# Export dataframe to a feather format file
df_result.reset_index().to_feather('results_v3.feather')